In [ ]:
import pandas as pd
import language_tool_python
from google_play_scraper import app
from bs4 import BeautifulSoup
import re
import numpy as np

In [ ]:
def clean_text(text):
    
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text(separator=" ")

    cleaned_text = re.sub(r"Ã¢Å|[^\w\s.,/'-`!?\’]", "", cleaned_text)
    
    cleaned_text = re.sub(r"\s+", " ", cleaned_text).strip()
    
    return cleaned_text.replace("WiFi", "Wi-Fi").replace("wifi", "wi-fi").replace("Wifi", "Wi-fi").replace("wiFi", "wi-Fi")

In [ ]:
tool = language_tool_python.LanguageTool('en-US')  

In [ ]:
xlsx_file = 'grammar.xlsx'  
data = pd.read_excel(xlsx_file)
app_id_column = 'app id'  
description_column = 'description'   
edited_description_column = 'edited description'
mistakes_column = 'mistakes'  
word_count_column = 'word count'

In [ ]:
descriptions = []


for index, row in data.iterrows():
    app_id = row[app_id_column]
    try:
        result = app(app_id)
        description = result['description']
        descriptions.append(description)
        print(f'{app_id} is done')
    except Exception as e:
        print(f"App ID '{app_id}' not found. Skipping...")
        descriptions.append(np.nan)

data[description_column] = descriptions
data.to_excel(xlsx_file, index=False)

    


In [ ]:
data.dropna(subset=[description_column], inplace=True)
edited_descriptions = []
# print(descriptions)


for index, row in data.iterrows():
        description = row[description_column]
        cleaned_description = clean_text(description)
        edited_descriptions.append(cleaned_description)
        

data[edited_description_column] = edited_descriptions
data.to_excel(xlsx_file, index=False)

In [ ]:
def get_unique_mistakes(matches):
    unique_mistakes = set()
    for match in matches:
        mistake = match.ruleId
        replacement = match.replacements[0] if match.replacements else ''
        unique_mistakes.add((mistake, replacement))
    return unique_mistakes

In [ ]:
mistake_counts = []
word_counts= []

for index, row in data.iterrows():
    print(index)
    description = row[edited_description_column]
    matches = tool.check(description)
    unique_mistakes = get_unique_mistakes(matches)
    
    num_mistakes = len(unique_mistakes)
    mistake_counts.append(num_mistakes)

    num_words = len(description.split())
    word_counts.append(num_words)

data[mistakes_column] = mistake_counts
data[word_count_column] = word_counts

data.to_excel(xlsx_file, index=False)